### Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

import collections

## 1. Load Dataset

In [599]:
incidents_df = pd.read_csv("data/incidents.csv")

/var/folders/6w/gs5ftqnj599ct2q_7plmjnk00000gn/T/ipykernel_51874/1719764553.py:1: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents_df = pd.read_csv("data/incidents.csv")


In [ ]:
incidents_df.isna().sum() / incidents_df.shape[0] * 100

In [ ]:
def convert_dtype(x):
    if not x:
        return np.NaN
    try:
        return float(x)
    except:
        print(f"Wrong type (removed): {x}")
        return "syntactically wrong"

In [ ]:
incidents_df = pd.read_csv(
    "data/incidents.csv",
    converters={
        "n_participants_child": convert_dtype,
        "n_participants_teen": convert_dtype,
        "n_participants_adult": convert_dtype,
        "min_age_participants": convert_dtype,
        "avg_age_participants": convert_dtype,
        "max_age_participants": convert_dtype,
    },
)

In [ ]:
incidents_df.shape[0] - incidents_df.drop_duplicates().shape[0]

In [ ]:
incidents_df = incidents_df.drop_duplicates(ignore_index=True)

In [ ]:
poverty_df = pd.read_csv("data/povertyByStateYear.csv")

poverty_df

In [ ]:
district_house_df = pd.read_csv("data/year_state_district_house.csv")
district_house_df

In [ ]:
np.min(incidents_df["congressional_district"])

In [ ]:
np.max(incidents_df["congressional_district"])

## 2. Data Quality Assessment

### Syntactic accuracy

In [ ]:
wrong_type_vars = [
    "n_participants_child",
    "n_participants_teen",
    "n_participants_adult",
    "min_age_participants",
    "avg_age_participants",
    "max_age_participants",
]

for var in wrong_type_vars:
    print(var)
    print(
        incidents_df.loc[incidents_df[var] == "syntactically wrong"].shape[0]
        / incidents_df.shape[0]
        * 100
    )

Now I can change these error to NaN values.

In [ ]:
for var in wrong_type_vars:
    incidents_df.loc[incidents_df[var] == "syntactically wrong", var] = np.NaN

In [ ]:
incidents_df["participant_age_group1"].unique()

### Semantic Accuracy

In [ ]:
incidents_df.insert(
    1, "year", [int(incidents_df["date"][i][0:4]) for i in range(incidents_df.shape[0])]
)

There are a lot of incidents with a wrong date. For the moment, we consider these years as missing values, but we keep the information about the date, which will be useful to try to correct the year of the incident.

In [ ]:
print(
    incidents_df.loc[incidents_df["year"] > 2018, "year"].shape[0]
    / incidents_df.shape[0]
    * 100
)
print(
    incidents_df.loc[incidents_df["year"] < 2013, "year"].shape[0]
    / incidents_df.shape[0]
    * 100
)

In [ ]:
incidents_df.loc[incidents_df["year"] > 2018, "year"] = np.NaN
incidents_df.loc[incidents_df["year"] < 2013, "year"] = np.NaN

We also verify that in 2018 recordings stop on March 31st.

In [ ]:
tdf = incidents_df.loc[incidents_df["year"] == 2018]
tdf.sort_values("date").tail()

We check if there are negative values for variables that we expect to be non-negative. 
We also want to remove improbable (too large) age values. We symbolically consider the maximum acceptable age to be 116, which is the oldest man on Earth's age.

In [ ]:
non_negative_vars = [
    "congressional_district",
    "state_house_district",
    "state_senate_district",
    "participant_age1",
    "min_age_participants",
    "avg_age_participants",
    "max_age_participants",
    "n_participants_child",
    "n_participants_teen",
    "n_participants_adult",
    "n_males",
    "n_females",
    "n_killed",
    "n_injured",
    "n_arrested",
    "n_unharmed",
    "n_participants",
]

age_vars = [
    "participant_age1",
    "min_age_participants",
    "avg_age_participants",
    "max_age_participants",
]

for var in non_negative_vars:
    print(var)
    tmp = incidents_df.loc[incidents_df[var].notna()]
    if var in age_vars:
        print(
            tmp.loc[tmp[var] < 0].shape[0] / incidents_df.shape[0] * 100
            + tmp.loc[tmp[var] > 116].shape[0] / incidents_df.shape[0] * 100
        )
        incidents_df.loc[incidents_df[var] < 0, var] = np.NaN
        incidents_df.loc[incidents_df[var] > 116, var] = np.NaN
    else:
        print(tmp.loc[tmp[var] < 0].shape[0] / incidents_df.shape[0] * 100)
        incidents_df.loc[incidents_df[var] < 0, var] = np.NaN

Other variables that should be constrained in a specific range are coordinates. Now, to be precise we should check that all coordinates match the respsective county or city, but here we just investigate cases where coordinates are not in the USA.

In [ ]:
print(np.min(incidents_df["latitude"]))
print(np.max(incidents_df["latitude"]))
print(np.min(incidents_df["longitude"]))
print(np.max(incidents_df["longitude"]))

In [ ]:
incidents_df.loc[incidents_df["longitude"] > -60]

In [ ]:
incidents_df.loc[incidents_df["longitude"] > -60, "longitude"] = np.NaN

Fortunately, there are only 5 cases where the longitude attribute seems wrong. So we can try to check if changing the sign of the longitude is consistent with the rest of the information on the location.

Using google maps, we verify that all the 5 locations match if the lognitude's sign is reverse. Moreover, these 5 incidents are all TSA Actions in different airports.

We can now correct the coordinates.

In [ ]:
incidents_df.loc[incidents_df["longitude"] > -60, "longitude"] = -incidents_df.loc[
    incidents_df["longitude"] > -60, "longitude"
]

In [ ]:
tmp_notna = incidents_df.loc[incidents_df["min_age_participants"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["max_age_participants"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["avg_age_participants"].notna()]

tmp = tmp_notna.loc[
    tmp_notna["max_age_participants"] >= tmp_notna["avg_age_participants"]
]
tmp = tmp.loc[tmp["min_age_participants"] <= tmp["avg_age_participants"]]

tmp.shape[0] / tmp_notna.shape[0] * 100

In [ ]:
tmp_notna = incidents_df.loc[incidents_df["min_age_participants"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["max_age_participants"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["participant_age1"].notna()]

tmp = tmp_notna.loc[tmp_notna["max_age_participants"] >= tmp_notna["participant_age1"]]
tmp = tmp.loc[tmp["min_age_participants"] <= tmp["participant_age1"]]

tmp.shape[0] / tmp_notna.shape[0] * 100

Now we check that the number of a special categrory of participants if not larger than the total number of participants.

In [ ]:
for col in [
    "n_killed",
    "n_injured",
    "n_unharmed",
    "n_arrested",
    "n_participants_child",
    "n_participants_teen",
    "n_participants_adult",
    "n_males",
    "n_females",
]:
    print(col)
    tmp_notna = incidents_df.loc[incidents_df[col].notna()]
    tmp = tmp_notna.loc[tmp_notna[col] > tmp_notna["n_participants"]]

    print(tmp.shape[0] / tmp_notna.shape[0] * 100)

    incidents_df[col][tmp.index] = np.NaN

Another thing that we may want to check is the information about the number of participants.
We expect the fllowing equalities to hold:
* n_participants = n_males + n_females 
* n_participants = n_participants_child + n_participants_teen + n_participants_adult
* n_participants = n_killed + n_unharmed + n_injured 
or
* n_participants = n_killed + n_unharmed + n_injured + n_arrested

In [ ]:
tmp_notna = incidents_df.loc[incidents_df["n_killed"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_injured"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_unharmed"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_arrested"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_participants"].notna()]

tmp = tmp_notna.loc[
    tmp_notna["n_killed"]
    + tmp_notna["n_injured"]
    + tmp_notna["n_unharmed"]
    + tmp_notna["n_arrested"]
    == tmp_notna["n_participants"]
]

tmp.shape[0] / tmp_notna.shape[0] * 100

In [ ]:
tmp_notna = incidents_df.loc[incidents_df["n_killed"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_injured"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_unharmed"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_arrested"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_participants"].notna()]

tmp_notna = tmp_notna.loc[
    tmp_notna["n_killed"]
    + tmp_notna["n_injured"]
    + tmp_notna["n_unharmed"]
    + tmp_notna["n_arrested"]
    != tmp_notna["n_participants"]
]

tmp = tmp_notna.loc[
    tmp_notna["n_killed"] + tmp_notna["n_injured"] + tmp_notna["n_unharmed"]
    == tmp_notna["n_participants"]
]

tmp.shape[0] / tmp_notna.shape[0] * 100

In [627]:
tmp_notna

,date,year,state,city_or_county,address,latitude,longitude,congressional_district,state_house_district,state_senate_district,...,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,notes,incident_characteristics1,incident_characteristics2
3,2016-10-15,2016.0,District of Columbia,Washington,"1000 block of Bladensburg Road, NE",38.9030,-76.9820,1.0,NaN,NaN,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,NaN,Shot - Wounded/Injured,NaN
35,2029-11-02,NaN,Illinois,Rockford,East State and Bell School Road,42.2704,-88.9703,16.0,68.0,34.0,...,1.0,0.0,0.0,0.0,0.0,1.0,2.0,Hoffman House and Scoreboard sports bar - Robb...,Defensive Use,Institution/Group/Business
37,2017-05-25,2017.0,Arkansas,Little Rock,3100 block of Boyle Park Road,34.7219,-92.3574,2.0,34.0,31.0,...,1.0,0.0,0.0,0.0,0.0,1.0,4.0,NaN,Shots Fired - No Injuries,Armed robbery with injury/death and/or evidenc...
140,2029-07-03,NaN,California,San Dimas,865 W. Arrow Highway,34.1067,-117.8240,32.0,41.0,25.0,...,1.0,0.0,0.0,0.0,0.0,1.0,2.0,at Check Into Cash,Armed robbery with injury/death and/or evidenc...,Institution/Group/Business
174,2014-05-02,2014.0,South Carolina,Murrells Inlet,5190 Highway 17 Bypass,33.5353,-79.0543,7.0,108.0,34.0,...,2.0,0.0,0.0,0.0,0.0,2.0,3.0,Not 2 Shabby,Armed robbery with injury/death and/or evidenc...,Institution/Group/Business
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239158,2015-08-04,2015.0,Indiana,Indianapolis,1400 block of Hamilton Avenue,39.7860,-86.1258,7.0,96.0,34.0,...,5.0,0.0,0.0,3.0,3.0,0.0,5.0,2 men stable after drug robbery/attack; 1 shoo...,Shot - Wounded/Injured,Drug involvement
239242,2014-09-07,2014.0,Indiana,Richmond,416 S. Ninth St.,39.8217,-84.8904,6.0,56.0,27.0,...,1.0,0.0,0.0,0.0,0.0,1.0,2.0,Family Dollar - robbed at gunpoint; suspect ap...,Drug involvement,Armed robbery with injury/death and/or evidenc...
239246,2016-09-16,2016.0,California,Bakersfield,2340 Niles Street,35.3760,-118.9650,23.0,32.0,16.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,"attempt to rob marijuana dispensary, shots exc...",Shot - Wounded/Injured,Institution/Group/Business
239266,2016-06-20,2016.0,Ohio,Columbus,4700 block of Lemarie Place,39.9612,-82.9988,3.0,18.0,15.0,...,1.0,0.0,0.0,1.0,0.0,0.0,3.0,"1 wounded, head, when 2 men fired on him in pa...",Shot - Wounded/Injured,NaN


In [628]:
tmp = tmp_notna.loc[
    tmp_notna["n_killed"]
    + tmp_notna["n_injured"]
    + tmp_notna["n_unharmed"]
    + tmp_notna["n_arrested"]
    != tmp_notna["n_participants"]
]
for i in tmp.index:
    tot = (
        incidents_df["n_killed"][i]
        + incidents_df["n_injured"][i]
        + incidents_df["n_arrested"][i]
        + incidents_df["n_unharmed"][i]
    )
    if tot > 0:
        for var in ["n_killed", "n_injured", "n_arrested"]:
            incidents_df[var][i] = int(
                (incidents_df[var][i] / tot) * incidents_df["n_participants"][i]
            )
        incidents_df["n_unharmed"][i] = (
            incidents_df["n_participants"][i]
            - incidents_df["n_killed"][i]
            - incidents_df["n_injured"][i]
            - incidents_df["n_arrested"][i]
        )
    else:
        incidents_df["n_killed"][i] = np.NaN
        incidents_df["n_injured"][i] = np.NaN
        incidents_df["n_arrested"][i] = np.NaN
        incidents_df["n_unharmed"][i] = np.NaN

In [ ]:
tmp_notna = incidents_df.loc[incidents_df["n_participants_child"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_participants_teen"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_participants_adult"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_participants"].notna()]

tmp = tmp_notna.loc[
    tmp_notna["n_participants_child"]
    + tmp_notna["n_participants_teen"]
    + tmp_notna["n_participants_adult"]
    == tmp_notna["n_participants"]
]

tmp.shape[0] / tmp_notna.shape[0] * 100

In [ ]:
tmp = tmp_notna.loc[
    tmp_notna["n_participants_child"]
    + tmp_notna["n_participants_teen"]
    + tmp_notna["n_participants_adult"]
    != tmp_notna["n_participants"]
]
for i in tmp.index:
    tot = (
        incidents_df["n_participants_child"][i]
        + incidents_df["n_participants_teen"][i]
        + incidents_df["n_participants_adult"][i]
    )
    if tot > 0:
        for var in ["n_participants_child", "n_participants_teen"]:
            incidents_df[var][i] = int(
                (incidents_df[var][i] / tot) * incidents_df["n_participants"][i]
            )
        incidents_df["n_participants_adult"][i] = (
            incidents_df["n_participants"][i]
            - incidents_df["n_participants_child"][i]
            - incidents_df["n_participants_teen"][i]
        )
    else:
        tmp_notna["n_participants_child"][i] = np.NaN
        tmp_notna["n_participants_teen"][i] = np.NaN
        tmp_notna["n_participants_adult"][i] = np.NaN

In [ ]:
tmp_notna = incidents_df.loc[incidents_df["n_males"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_females"].notna()]
tmp_notna = tmp_notna.loc[tmp_notna["n_participants"].notna()]

tmp = tmp_notna.loc[
    tmp_notna["n_males"] + tmp_notna["n_females"] == tmp_notna["n_participants"]
]

tmp.shape[0] / tmp_notna.shape[0] * 100

In [ ]:
tmp = tmp_notna.loc[
    tmp_notna["n_males"] + tmp_notna["n_females"] != tmp_notna["n_participants"]
]

for i in tmp.index:
    tot = incidents_df["n_males"][i] + incidents_df["n_females"][i]
    if tot > 0:
        incidents_df["n_males"][i] = int(
            (incidents_df["n_males"][i] / tot) * incidents_df["n_participants"][i]
        )
        incidents_df["n_females"][i] = (
            incidents_df["n_participants"][i] - incidents_df["n_males"][i]
        )
    else:
        incidents_df["n_males"][i] = np.NaN
        incidents_df["n_females"][i] = np.NaN

Even if the count is not always correct, for now we keep the information about the group composition as it could still be useful. Also because it would be difficult to identify the incorrect value among the different attributes in the sum.

In [ ]:
incidents_df.loc[incidents_df["n_participants"] == 0].shape[0] / incidents_df.shape[
    0
] * 100

In [ ]:
for var in [
    "n_males",
    "n_females",
    "n_killed",
    "n_injured",
    "n_unharmed",
    "n_arrested",
    "n_participants_adult",
    "n_participants_teen",
    "n_participants_child",
]:
    print(
        incidents_df.loc[
            (incidents_df["n_participants"] == 0) & (incidents_df[var] > 0)
        ].shape[0]
    )

In [ ]:
incidents_df.loc[incidents_df["n_participants"] == 0, "n_participants"] = np.NaN

In [ ]:
(incidents_df.isnull().sum() / incidents_df.shape[0]) * 100

### Data Integration

In [ ]:
joined_df = incidents_df.copy()

In [ ]:
joined_df["povertyPercentage"] = " "
joined_df["party"] = " "
joined_df["candidatevotes"] = " "
joined_df["totalvotes"] = " "

valid dates: 1/1/2013 to 31/3/2018. \
there are a lot of incidents with wrong dates (2028-2030). \
Ho googlato un po' di notizie usando le note del dataset, le città ecc. e sembra che 2028->2013; 2029->2014; 2030->2015.

In [ ]:
pd.options.mode.chained_assignment = None

for i in range(joined_df.shape[0]):
    if int(joined_df["date"][i][0:4]) > 2018:
        joined_df["year"][i] = int(joined_df["date"][i][0:4]) - 15

In [ ]:
for i in range(joined_df.shape[0]):
    povertyPercentage = poverty_df.loc[
        (poverty_df["year"] == joined_df.loc[i, "year"])
        & (poverty_df["state"] == joined_df.loc[i, "state"]),
        "povertyPercentage",
    ].values

    if len(povertyPercentage) == 1:
        joined_df["povertyPercentage"][i] = povertyPercentage[0]
    else:
        joined_df["povertyPercentage"][i] = np.NaN

Congressional elections occur every 2 years!

In [ ]:
for i in range(joined_df.shape[0]):
    party = district_house_df.loc[
        (district_house_df["year"] // 2 == joined_df["year"][i] // 2)
        & (district_house_df["state"] == joined_df["state"][i].upper())
        & (
            district_house_df["congressional_district"]
            == joined_df["congressional_district"][i]
        ),
        "party",
    ].values

    if len(party) == 1:
        joined_df["party"][i] = party[0]
    else:
        joined_df["party"][i] = " "

In [ ]:
for i in range(joined_df.shape[0]):
    candidatevotes = district_house_df.loc[
        (district_house_df["year"] // 2 == joined_df.loc[i, "year"] // 2)
        & (district_house_df["state"] == joined_df.loc[i, "state"].upper())
        & (
            district_house_df["congressional_district"]
            == joined_df.loc[i, "congressional_district"]
        ),
        "candidatevotes",
    ].values

    if len(candidatevotes) == 1:
        joined_df["candidatevotes"][i] = candidatevotes[0]
    else:
        joined_df["candidatevotes"][i] = np.NaN

In [ ]:
for i in range(joined_df.shape[0]):
    totalvotes = district_house_df.loc[
        (district_house_df["year"] // 2 == joined_df.loc[i, "year"] // 2)
        & (district_house_df["state"] == joined_df.loc[i, "state"].upper())
        & (
            district_house_df["congressional_district"]
            == joined_df.loc[i, "congressional_district"]
        ),
        "totalvotes",
    ].values

    if len(totalvotes) == 1:
        joined_df["totalvotes"][i] = totalvotes[0]
    else:
        joined_df["totalvotes"][i] = np.NaN

In [ ]:
joined_df.head()

In [ ]:
joined_df.to_csv("data/joined_dataset.csv", index=False)

## 3. Data preparation

### Fill missing values

In [ ]:
joined_df = pd.read_csv("data/joined_dataset.csv")

In [ ]:
def fill_missing_randomly_within_group(df, value_column, group_columns=[]):
    if len(group_columns) > 0:
        grouped = df.groupby(group_columns)
        for _, group_df in grouped:
            non_missing_values = group_df[value_column].dropna()
            missing_indices = group_df.index[
                group_df[value_column].isnull().any(axis=1)
            ]
            if non_missing_values.shape[0] > 0:
                random_indexes = np.random.choice(
                    [i for i in non_missing_values.index],
                    size=len(missing_indices),
                )
                for var in value_column:
                    df.loc[missing_indices, var] = df.loc[random_indexes, var].values
    else:
        non_missing_values = df[value_column].dropna()
        missing_indices = df.index[df[value_column].isnull().any(axis=1)]
        if non_missing_values.shape[0] > 0:
            random_indexes = np.random.choice(
                [i for i in non_missing_values.index],
                size=len(missing_indices),
            )
            for var in value_column:
                df.loc[missing_indices, var] = df.loc[random_indexes, var].values

    return df

#### 1. Coordinates

In [ ]:
joined_df = fill_missing_randomly_within_group(
    joined_df, ["latitude", "longitude"], ["state", "city_or_county"]
)

If we have no info about city, we use states:

In [ ]:
joined_df = fill_missing_randomly_within_group(
    joined_df, ["latitude", "longitude"], ["state"]
)

#### 2. Age

In [ ]:
joined_df = fill_missing_randomly_within_group(
    joined_df,
    ["min_age_participants", "avg_age_participants", "max_age_participants"],
    ["state"],
)

#### 3. Participants

In [ ]:
joined_df = fill_missing_randomly_within_group(
    joined_df, ["n_participants"], ["state", "year"]
)

males/females

In [ ]:
joined_df["n_males"] = joined_df["n_males"].div(
    (joined_df["n_participants"]).replace(0, 1)
)

joined_df = fill_missing_randomly_within_group(joined_df, ["n_males"], ["state"])

joined_df["n_males"] = (joined_df["n_males"] * joined_df["n_participants"]).astype(int)
joined_df["n_females"] = (joined_df["n_participants"] - joined_df["n_males"]).astype(
    int
)

adults/teen/children

In [ ]:
joined_df["n_participants_adult"] = joined_df["n_participants_adult"].div(
    (joined_df["n_participants"]).replace(0, 1)
)
joined_df["n_participants_teen"] = joined_df["n_participants_teen"].div(
    (joined_df["n_participants"]).replace(0, 1)
)

joined_df = fill_missing_randomly_within_group(
    joined_df, ["n_participants_adult", "n_participants_teen"], ["state"]
)

joined_df["n_participants_adult"] = (
    joined_df["n_participants_adult"] * joined_df["n_participants"]
).astype(int)
joined_df["n_participants_teen"] = (
    joined_df["n_participants_teen"] * joined_df["n_participants"]
).astype(int)
joined_df["n_participants_child"] = (
    joined_df["n_participants"]
    - joined_df["n_participants_adult"]
    - joined_df["n_participants_teen"]
).astype(int)

killed/injured/arrested/unharmed

In [ ]:
joined_df["n_killed"] = joined_df["n_killed"].div(
    (joined_df["n_participants"]).replace(0, 1)
)
joined_df["n_injured"] = joined_df["n_injured"].div(
    (joined_df["n_participants"]).replace(0, 1)
)
joined_df["n_arrested"] = joined_df["n_arrested"].div(
    (joined_df["n_participants"]).replace(0, 1)
)

joined_df = fill_missing_randomly_within_group(
    joined_df, ["n_killed", "n_injured", "n_arrested", "n_unharmed"], ["state"]
)

joined_df["n_killed"] = (joined_df["n_killed"] * joined_df["n_participants"]).astype(
    int
)
joined_df["n_injured"] = (joined_df["n_injured"] * joined_df["n_participants"]).astype(
    int
)
joined_df["n_arrested"] = (
    joined_df["n_arrested"] * joined_df["n_participants"]
).astype(int)

joined_df["n_unharmed"] = (
    joined_df["n_unharmed"] * joined_df["n_participants"]
).astype(int)

#### 4. Party and votes

In [ ]:
joined_df.loc[joined_df["party"] == " ", "party"] = np.NaN
joined_df.loc[joined_df["totalvotes"] == 0, "totalvotes"] = np.NaN
joined_df.loc[joined_df["candidatevotes"] == 0, "candidatevotes"] = np.NaN

In [ ]:
joined_df = fill_missing_randomly_within_group(
    joined_df,
    ["party", "totalvotes", "candidatevotes"],
    ["state", "congressional_district"],
)
joined_df = fill_missing_randomly_within_group(
    joined_df,
    ["party", "totalvotes", "candidatevotes"],
    ["state", "city_or_county", "year"],
)
joined_df = fill_missing_randomly_within_group(
    joined_df, ["party", "totalvotes", "candidatevotes"], ["state", "year"]
)

joined_df = fill_missing_randomly_within_group(
    joined_df, ["party", "totalvotes", "candidatevotes"], ["year"]
)

In [ ]:
print(joined_df[joined_df["party"].isna()].shape[0] / joined_df.shape[0])
print(joined_df[joined_df["totalvotes"].isna()].shape[0] / joined_df.shape[0])
print(joined_df[joined_df["candidatevotes"].isna()].shape[0] / joined_df.shape[0])

### Remove / modify attributes

In [ ]:
joined_df = joined_df.drop(
    columns=[
        "address",
        "congressional_district",
        "state_house_district",
        "state_senate_district",
        "participant_age1",
        "participant_age_group1",
        "participant_gender1",
        "notes",
        "incident_characteristics1",
        "incident_characteristics2",
    ]
)

In [ ]:
joined_df.insert(
    1,
    "month",
    [int(joined_df["date"][i][5:7]) for i in range(joined_df.shape[0])],
)

In [ ]:
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

def compute_progressive_date(date):
    """Compute the progressive date of the incident, starting from 01/01/2013.
    Data in "date" column are in the format "YYYY-MM-DD".

    Args:
        x (str): date of the incident

    Returns:
        int: progressive date of the incident
    """
    return (int(date[0:4])) * 365 + days_per_month[int(date[5:7]) - 1] + int(date[8:10])

for i in range(joined_df.shape[0]):
    joined_df["date"][i] = compute_progressive_date(joined_df["date"][i])

    # Incidents are between 2013 and 2017, the only bisestile 2016:
    if (joined_df["year"][i] == 2016 and joined_df["month"][i] >= 3) or joined_df["year"][i] > 2016:
        joined_df["date"][i] = joined_df["date"][i] + 1

start_date = joined_df["date"].min()
joined_df["date"] = joined_df["date"] - start_date

In [ ]:
joined_df.loc[joined_df["party"] == "DEMOCRATIC-FARMER-LABOR", "party"] = "DEMOCRAT"

In [ ]:
joined_df["republicans_ratio"] = " "
joined_df["democrats_ratio"] = " "

for i in joined_df.index:
    if joined_df["party"][i] == "REPUBLICAN":
        joined_df["republicans_ratio"][i] = (
            joined_df["candidatevotes"][i] / joined_df["totalvotes"][i]
        )
        joined_df["democrats_ratio"][i] = 1 - joined_df["republicans_ratio"][i]
    else:
        joined_df["democrats_ratio"][i] = (
            joined_df["candidatevotes"][i] / joined_df["totalvotes"][i]
        )
        joined_df["republicans_ratio"][i] = 1 - joined_df["democrats_ratio"][i]

In [ ]:
joined_df.drop(["candidatevotes"], axis=1, inplace=True)

In [ ]:
joined_df.isna().sum() / joined_df.shape[0] * 100

In [ ]:
joined_df[
    [
        "min_age_participants",
        "max_age_participants",
        "n_participants_child",
        "n_participants_teen",
        "n_participants_adult",
        "povertyPercentage",
    ]
] = joined_df[
    [
        "min_age_participants",
        "max_age_participants",
        "n_participants_child",
        "n_participants_teen",
        "n_participants_adult",
        "povertyPercentage",
    ]
].astype(
    float
)

In [ ]:
columns_to_normalize = [
    "n_arrested",
    "n_unharmed",
    "n_killed",
    "n_injured",
    "n_participants_adult",
    "n_participants_teen",
    "n_participants_child",
    "n_males",
    "n_females",
]

# Normalize columns by dividing each element by n_participants
joined_df[columns_to_normalize] = joined_df[columns_to_normalize].div(
    joined_df["n_participants"], axis=0
)

joined_df = joined_df.rename(
    columns={
        "n_arrested": "arrested_ratio",
        "n_unharmed": "unharmed_ratio",
        "n_killed": "killed_ratio",
        "n_injured": "injured_ratio",
        "n_participants_adult": "adults_ratio",
        "n_participants_teen": "teen_ratio",
        "n_participants_child": "children_ratio",
        "n_males": "males_ratio",
        "n_females": "females_ratio",
    }
)

### Additional information

In [ ]:
population_df = pd.read_csv("data/population.csv")
population_df = population_df[["placeName", "Date:Count_Person", "Value:Count_Person"]]
population_df = population_df.astype({})
population_df = population_df.rename(
    columns={
        "placeName": "state",
        "Date:Count_Person": "year",
        "Value:Count_Person": "population",
    }
)

In [ ]:
joined_df["state_population"] = " "

for i in range(joined_df.shape[0]):
    year_condition = population_df["year"] == joined_df["year"][i]
    state_condition = population_df["state"] == joined_df["state"][i]
    population = population_df.loc[
        year_condition & state_condition, "population"
    ].values

    if len(population) == 1:
        joined_df["state_population"][i] = population[0]
    else:
        joined_df["state_population"][i] = np.NaN

Introduce a new column in dataframe for indicating whether a city is populous or not.

In [ ]:
very_pop_cities = [
    "New York",
    "Los Angeles",
    "Chicago",
    "Houston",
    "Phoenix",
    "Philadelphia",
    "San Antonio",
    "San Diego",
    "Dallas",
    "San Jose",
    "Austin",
    "Jacksonville",
    "San Francisco",
    "Columbus",
    "Fort Worth",
    "Indianapolis",
    "Charlotte",
    "Seattle",
    "Denver",
    "Washington",
    "Boston",
    "El Paso",
    "Detroit",
    "Nashville",
    "Memphis",
    "Portland",
    "Oklahoma City",
    "Las Vegas",
    "Louisville",
    "Baltimore",
    "Milwaukee",
    "Albuquerque",
    "Tucson",
]

In [ ]:
joined_df["populous_city"] = " "    

count = 0
for i in joined_df.index:
    if joined_df["populous_city"][i] in very_pop_cities:
        joined_df["populous_city"][i] = 1
        count = count + 1
    else:
        joined_df["populous_city"][i] = 0

print(count)

## 4. Variables distribution

### Party

In [ ]:
sns.set_style("whitegrid")

colors = {
    "DEMOCRAT": "blue",
    "DEMOCRATIC-FARMER-LABOR": "purple",
    "REPUBLICAN": "red",
    " ": "gray",
}
usa = plt.imread("images/NorthAmerica.png")

fig, axs = plt.subplots(2, 2, figsize=(20, 9))
for i, y in enumerate([2012, 2014, 2016, 2018]):
    axs[i // 2, i % 2].imshow(usa, extent=[-180, -60, 20, 80])
    year_df = joined_df.loc[joined_df["year"] // 2 == y // 2]
    if y == 2012:
        axs[i // 2, i % 2].set_title(f"2013", fontsize=20)
    elif y == 2018:
        axs[i // 2, i % 2].set_title(f"2018", fontsize=20)
    else:
        axs[i // 2, i % 2].set_title(f"{y}-{y+1}", fontsize=20)
    axs[i // 2, i % 2].scatter(
        [year_df["longitude"][i] for i in year_df.index],
        [year_df["latitude"][i] for i in year_df.index],
        s=3,
        c=[colors[year_df["party"][i]] for i in year_df.index],
    )
    axs[i // 2, i % 2].set_xlim([-180, -60])
    axs[i // 2, i % 2].set_ylim([18, 75])
    axs[i // 2, i % 2].plot([0], "r", label="REPUBLICAN")
    axs[i // 2, i % 2].plot([0], "b", label="DEMOCRAT")
    axs[i // 2, i % 2].legend()

    sns.set(font_scale=1)

### Population

In [ ]:
import random
import matplotlib.colors as mcolors

# Generate a list of 50 random RGBA colors
random_colors = [
    (random.random(), random.random(), random.random(), random.uniform(0.5, 1.0))
    for _ in range(51)
]

colors = {}
state_list = list(joined_df.sort_values("state", ascending=False)["state"].unique())
for i, state in enumerate(state_list):
    colors[state] = random_colors[i]

import random

In [ ]:
sns.set_style("whitegrid")

usa = plt.imread("images/NorthAmerica.png")

fig, axs = plt.subplots(
    1,
    2,
    figsize=(30, 10),
    gridspec_kw={"height_ratios": [1], "width_ratios": [1, 1]},
)

axs[0].imshow(usa, extent=[-180, -60, 20, 80])
axs[0].scatter(
    [joined_df["longitude"][i] for i in joined_df.index],
    [joined_df["latitude"][i] for i in joined_df.index],
    s=1,
    c=[colors[joined_df["state"][i]] for i in joined_df.index],
)
axs[0].set_xlim([-170, -65])
axs[0].set_ylim([18, 80])

for i, value in enumerate(
    joined_df.sort_values("state", ascending=False)["state"].unique()
):
    count = joined_df.loc[joined_df["state"] == value].shape[0]
    axs[1].barh(value, count, color=random_colors[i])

axs[1].set_xlabel("Number of incidents from Jan 2013 to Mar 2018")
sns.set(font_scale=1.5)

### Poverty percentage

In [ ]:
import matplotlib.cm as cm

sns.set_style("whitegrid")

usa = plt.imread("images/NorthAmerica.png")

fig, axs = plt.subplots(
    1,
    2,
    figsize=(30, 10),
    gridspec_kw={"height_ratios": [1], "width_ratios": [1, 1]},
)

axs[0].imshow(usa, extent=[-180, -60, 20, 80])
scatter = axs[0].scatter(
    [joined_df["longitude"][i] for i in joined_df.index],
    [joined_df["latitude"][i] for i in joined_df.index],
    s=1,
    c=[joined_df["povertyPercentage"][i] for i in joined_df.index],
)

axs[0].set_xlim([-170, -60])
axs[0].set_ylim([18, 80])

####

states = joined_df.sort_values("state", ascending=False)["state"].unique()
incidents_rate = []

poverty = []
for i, value in enumerate(states):
    poverty.append(
        np.mean(joined_df.loc[joined_df["state"] == value]["povertyPercentage"])
    )

sorted_indices = np.argsort(poverty)

# Use sorted indices to reorder both arrays
poverty = [poverty[i] for i in sorted_indices]
states = [states[i] for i in sorted_indices]

cmap = cm.magma
norm = plt.Normalize(min(poverty), max(poverty))
colors = cmap(norm(poverty))
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=axs[1], label="Poverty percentage")

for i, value in enumerate(states):
    count = joined_df.loc[joined_df["state"] == value].shape[0]
    population = population_df[
        (population_df["state"] == value) & (population_df["year"] == 2018)
    ]["population"]
    incidents_rate.append(count / population * 1e6)
    axs[1].barh(value, count / population * 1e6, color=colors[i])

axs[1].set_xlabel("Number of incidents every million people from Jan 2013 to Mar 2018")
sns.set(font_scale=0.5)

## 5. Remove outliers

In [ ]:
numerical_columns = [
    "date",
    "latitude",
    "longitude",
    "min_age_participants",
    "avg_age_participants",
    "max_age_participants",
    "children_ratio",
    "teen_ratio",
    "adults_ratio",
    "males_ratio",
    "females_ratio",
    "killed_ratio",
    "injured_ratio",
    "arrested_ratio",
    "unharmed_ratio",
    "n_participants",
    "povertyPercentage",
    "republicans_ratio",
    "democrats_ratio",
    "totalvotes",
    "dead_murder_or_suicide",
]

In [ ]:
sns.set_style("whitegrid")
# Compute the correlation matrix
corr = joined_df[numerical_columns].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

sns.set(font_scale=0.8)

In [ ]:
joined_df.drop("females_ratio", axis=1, inplace=True)
joined_df.drop("children_ratio", axis=1, inplace=True)

## 6. Summary

In [ ]:
final_df = joined_df.copy()

In [ ]:
final_df.isna().sum() / joined_df.shape[0] * 100

Encodings

In [ ]:
final_df = pd.get_dummies(final_df, columns=["state"])

In [ ]:
final_df.loc[final_df["party"] == "DEMOCRAT", "party"] = 0
final_df.loc[final_df["party"] == "REPUBLICAN", "party"] = 1

In [ ]:
final_df["month_x"] = np.sin(final_df["month"] * 2 * np.pi / 12)
final_df["month_y"] = np.cos(final_df["month"] * 2 * np.pi / 12)
final_df.drop("month", axis=1, inplace=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv("data/final_dataset.csv", index=False)